Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Build an ML Pipeline

In this notebook, you learn how to create a machine learning training pipeline by using Azure Machine Learning components.

1. Prepare and create components into the workspace.
2. Use the component and pipeline SDK to create a pipeline the registered components.

## Prerequisites
* Install azure-ai-ml sdk following the [instructions here](../../README.md).
* Initialize credential & create compute clusters following [instructions here](../../configuration.ipynb);

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [3]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import load_component, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import MLClient
from azure.ai.ml.constants import AssetTypes, InputOutputModes, InputOutputModes

import os

# enable internal components in v2
os.environ["AZURE_ML_INTERNAL_COMPONENTS_ENABLED"] = "True"

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [4]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot.this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	AzureDeveloperCliCredential: Azure Developer CLI could not be found. Please visit https://aka.ms/azure-dev for installation instructions and then,once installed, authenticate to your Azure account using 'azd login'.
	SharedTokenCacheCredential: Azure Active Directory error '(invalid_grant) AADSTS700082: The refresh token has expired due to inactivity. The token was issued on 2022-11-21T03:07:07.6802296Z and was inactive for 90.00:00:00.
Trace ID: b011f9b9-1903-4c5b-85c2-db0dbedc1900
Correlation ID: 5861cdf5-1c2e-4f70-b8bc-3114a4e947ae
Timestam

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [5]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

Found the config file in: D:\programs\azureml-examples\sdk\.azureml\config.json


enable_node_public_ip: true
id: /subscriptions/96aede12-2f73-41cb-b983-6d11a904839b/resourceGroups/hod-eastus2/providers/Microsoft.MachineLearningServices/workspaces/sdk_vnext_cli/computes/cpu-cluster
idle_time_before_scale_down: 120
location: eastus2
max_instances: 4
min_instances: 0
name: cpu-cluster
provisioning_state: Succeeded
size: STANDARD_DS2_V2
ssh_public_access_enabled: true
tier: dedicated
type: amlcompute



In [2]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml.dsl import pipeline
from sdk.create_and_link import get_my_data, consume_data

input_asset = Input(
    type="uri_file",
    path="azureml://datastores/workspaceblobstore/paths/miguTestMLTable/relative_path_in_content.txt",
)

In [12]:
@pipeline
def my_pipeline():
    node_0 = get_my_data()
    data_input = Input(type="uri_folder", path=node_0.outputs.output)
    # Input(type="uri_file", path="https://dprepdata.blob.core.windows.net/demo/Titanic.csv")
    node_1 = consume_data(data_path=data_input)

# create a pipeline
pipeline_job = my_pipeline()
pipeline_job.settings.default_compute = "cpu-cluster"
# pipeline_job.settings.default_datastore = "workspaceblobstore"
# pipeline_job.settings._dataset_access_mode = "DatasetInDpv2"

# Validating the pipeline
ml_client.jobs.validate(pipeline_job)

{
  "result": "Succeeded"
}

In [13]:
# Specify the workspace for workspace independent component when submitting the pipeline.
created_pipeline_job = ml_client.jobs.create_or_update(pipeline_job, experiment_name="hod_test_mltable")

# show detail information of run
created_pipeline_job

HttpResponseError: (UserError) Invalid binding expression since the source componentJob None does not exist.
Code: UserError
Message: Invalid binding expression since the source componentJob None does not exist.
Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "2a6d0a804ee02f66c6bf6074b786fa3c",
        "request": "c645f0c0524a10f0"
    }
}Type: Environment
Info: {
    "value": "eastus2"
}Type: Location
Info: {
    "value": "eastus2"
}Type: Time
Info: {
    "value": "2023-02-22T11:18:37.4547875+00:00"
}Type: InnerError
Info: {
    "value": {
        "code": "BadArgument",
        "innerError": {
            "code": "ArgumentInvalid",
            "innerError": {
                "code": "InvalidPipelineJob",
                "innerError": {
                    "code": "SourceComponentJobNotExsit",
                    "innerError": null
                }
            }
        }
    }
}